In [1]:
import sys
import os
import json
import pandas as pd
import numpy as np
import optparse
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import  Dense, Dropout, Activation, Conv2D, Flatten
from keras.layers.pooling import MaxPooling2D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict
from sklearn.model_selection import train_test_split
import os, glob
import numpy as np
from keras.models import model_from_yaml
from keras.preprocessing import image
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
#a) Use the "ImageDataGenerator()" class
datagen = ImageDataGenerator(rescale = 1./255,
                            shear_range = .2,
                            zoom_range = .2,
                            horizontal_flip = True)

In [3]:
#b) 
train = datagen.flow_from_directory(directory = r"dataset_train",
                           target_size = (64, 64),
                            batch_size = 32,
                            class_mode = 'categorical'  )
test = datagen.flow_from_directory(directory = r"dataset_test",
                                   batch_size = 1,
                                   target_size = (64, 64),
                                  class_mode=None,
                                  shuffle=False)
print(test.filepaths)
print(test.directory)

Found 88 images belonging to 4 classes.
Found 8 images belonging to 1 classes.
['dataset_test\\New folder\\1022.png', 'dataset_test\\New folder\\1053.png', 'dataset_test\\New folder\\4011.png', 'dataset_test\\New folder\\4053.png', 'dataset_test\\New folder\\6023.png', 'dataset_test\\New folder\\6051.png', 'dataset_test\\New folder\\C014.png', 'dataset_test\\New folder\\C033.png']
dataset_test


In [4]:
#c) Take a look at your training set
print(train.image_shape)
print(train.classes)


(64, 64, 3)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


The Image shapes are 64 x 64 and are 3 layers deep. There are 3 classes we need to predict on.

In [5]:
#2. Initial Classifier Build:
classifier = Sequential()
classifier.add(Conv2D(filters = 32,
                    kernel_size = (3,3),
                    input_shape = (64,64,3),
                    activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(64,kernel_size = (3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(units=128,
                    activation='relu'))
classifier.add(Dense(units = 4,
                    activation = 'softmax'))
classifier.compile(optimizer='adam', 
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'] )


Instructions for updating:
Colocations handled automatically by placer.


In [6]:
#3)a) Use .fit_generator() 
classifier.fit_generator(train,steps_per_epoch = 10, epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
10/10 [==============================] - 6s 627ms/step - loss: 1.1133 - acc: 0.6269
Epoch 2/10
10/10 [==============================] - 6s 590ms/step - loss: 0.1990 - acc: 0.9540
Epoch 3/10
10/10 [==============================] - 6s 556ms/step - loss: 0.1299 - acc: 0.9482
Epoch 4/10
10/10 [==============================] - 6s 617ms/step - loss: 0.0645 - acc: 0.9783
Epoch 5/10
10/10 [==============================] - 6s 552ms/step - loss: 0.0342 - acc: 0.9865
Epoch 6/10
10/10 [==============================] - 5s 474ms/step - loss: 0.0389 - acc: 0.9804
Epoch 7/10
10/10 [==============================] - 5s 515ms/step - loss: 0.0161 - acc: 0.9928
Epoch 8/10
10/10 [==============================] - 5s 513ms/step - loss: 0.0113 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 6s 553ms/step - loss: 0.0079 - acc: 0.9968
Epoch 10/10
10/10 [==============================] - 6s 618ms/step - loss: 0.0053 - acc: 1.0000


In [7]:
#b) Write out each model & model_weights to a file. An example is below:
# write model and model weights to disk
model_yaml = classifier.to_yaml()
with open(r'C:\Users\u353822\Documents\GitHub\University-of-Chicago1\University of Chicago\Machine Learning\HW8\model_1.yaml', "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_1.h5")
print("Saved model to disk")

Saved model to disk


In [8]:
#c) Predict using the model built in step 2. An example below shows how to reread weights & model: 
# load model from disk
yaml_file = open(r'C:\Users\u353822\Documents\GitHub\University-of-Chicago1\University of Chicago\Machine Learning\HW8\model_1.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)
# load weights into new model
model.load_weights("model_1.h5")
print("Loaded model from disk")

Loaded model from disk


In [9]:
# test data path
img_dir = r"C:\Users\u353822\Documents\GitHub\University-of-Chicago1\University of Chicago\Machine Learning\HW8\dataset_test\New folder"
# Enter Directory of all images
# iterate over each test image
# make a prediction and add to results 
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)
data = []
results = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

results

[array([0], dtype=int64),
 array([0], dtype=int64),
 array([0], dtype=int64),
 array([2], dtype=int64),
 array([1], dtype=int64),
 array([1], dtype=int64),
 array([1], dtype=int64),
 array([3], dtype=int64)]

Category 1 looks like the letter n.
Category 2 looks like a black circle with white figures inside split up into 4 quadrants.
Category 3 looks like a house.
Category 4 looks like a cross.
The data in test belongs to the following classes respectively:
C1, C1, C3, C3, C2, C2, C4,C4. 
Our model produced the collowing results for test prediction respectively:
C1, C1, C4, C3, C2, C2, C2, C4.
Our model predicted 6/8 images correctly leading to a 75% accuracy

In [11]:
steps_per_epoch = [10,10,10,30,30,30,50,50,50,50]
epoch = [10,20,30,10,20,30,10,20,30,100]
df2 = []
test1=[0,0,2,2,1,1,3,3]
df1 = {}
for s in range(len(steps_per_epoch)):
    classifier.fit_generator(train,steps_per_epoch = steps_per_epoch[s], epochs=epoch[s])
    y_pred = classifier.predict_generator(test, steps=8)
    y_preds = []
    for x in range(8):
        y_preds.append(list(y_pred[x,:]).index(max(y_pred[x,:])))
    ACC = accuracy_score(test1,y_preds)
    
    df1[s] = {'Steps per Epoch' : steps_per_epoch[s], 'Epochs' : epoch[s], 'Accuracy' : ACC}
df = pd.DataFrame.from_dict(df1, orient = 'index').reset_index(drop = True) 
df2.append(df)  
df3 = pd.concat(df2,axis=0).reset_index(drop = True)

Epoch 1/10
10/10 [==============================] - 5s 461ms/step - loss: 0.0026 - acc: 1.0000
Epoch 2/10
10/10 [==============================] - 5s 464ms/step - loss: 0.0029 - acc: 1.0000
Epoch 3/10
10/10 [==============================] - 5s 461ms/step - loss: 0.0019 - acc: 1.0000
Epoch 4/10
10/10 [==============================] - 5s 497ms/step - loss: 0.0056 - acc: 0.9968
Epoch 5/10
10/10 [==============================] - 5s 532ms/step - loss: 0.0014 - acc: 1.0000
Epoch 6/10
10/10 [==============================] - 5s 519ms/step - loss: 0.0019 - acc: 1.0000
Epoch 7/10
10/10 [==============================] - 5s 527ms/step - loss: 0.0012 - acc: 1.0000
Epoch 8/10
10/10 [==============================] - 5s 517ms/step - loss: 0.0014 - acc: 1.0000
Epoch 9/10
10/10 [==============================] - 5s 490ms/step - loss: 8.4836e-04 - acc: 1.0000
Epoch 10/10
10/10 [==============================] - 4s 427ms/step - loss: 8.3330e-04 - acc: 1.0000
Epoch 1/20
10/10 [=======================

30/30 [==============================] - 14s 462ms/step - loss: 5.9080e-06 - acc: 1.0000
Epoch 14/20
30/30 [==============================] - 13s 448ms/step - loss: 9.1644e-06 - acc: 1.0000
Epoch 15/20
30/30 [==============================] - 15s 485ms/step - loss: 1.0237e-05 - acc: 1.0000
Epoch 16/20
30/30 [==============================] - 14s 466ms/step - loss: 9.6333e-06 - acc: 1.0000
Epoch 17/20
30/30 [==============================] - 14s 483ms/step - loss: 9.3968e-06 - acc: 1.0000
Epoch 18/20
30/30 [==============================] - 15s 490ms/step - loss: 1.0950e-05 - acc: 1.0000
Epoch 19/20
30/30 [==============================] - 15s 512ms/step - loss: 5.6235e-06 - acc: 1.0000
Epoch 20/20
30/30 [==============================] - 14s 469ms/step - loss: 4.6450e-06 - acc: 1.0000
Epoch 1/30
30/30 [==============================] - 15s 492ms/step - loss: 5.5399e-06 - acc: 1.0000
Epoch 2/30
30/30 [==============================] - 14s 470ms/step - loss: 4.0736e-06 - acc: 1.0000
Epoc

50/50 [==============================] - 26s 520ms/step - loss: 3.8161e-07 - acc: 1.0000
Epoch 15/30
50/50 [==============================] - 25s 494ms/step - loss: 4.5113e-07 - acc: 1.0000
Epoch 16/30
50/50 [==============================] - 24s 471ms/step - loss: 4.5238e-07 - acc: 1.0000
Epoch 17/30
50/50 [==============================] - 28s 558ms/step - loss: 4.0822e-07 - acc: 1.0000
Epoch 18/30
50/50 [==============================] - 24s 479ms/step - loss: 3.2389e-07 - acc: 1.0000
Epoch 19/30
50/50 [==============================] - 23s 454ms/step - loss: 3.8059e-07 - acc: 1.0000
Epoch 20/30
50/50 [==============================] - 22s 444ms/step - loss: 3.8991e-07 - acc: 1.0000
Epoch 21/30
50/50 [==============================] - 23s 457ms/step - loss: 3.7638e-07 - acc: 1.0000
Epoch 22/30
50/50 [==============================] - 23s 462ms/step - loss: 2.9942e-07 - acc: 1.0000
Epoch 23/30
50/50 [==============================] - 24s 471ms/step - loss: 3.7914e-07 - acc: 1.0000
Ep

50/50 [==============================] - 25s 508ms/step - loss: 1.4101e-07 - acc: 1.0000
Epoch 65/100
50/50 [==============================] - 23s 460ms/step - loss: 1.3606e-07 - acc: 1.0000
Epoch 66/100
50/50 [==============================] - 23s 462ms/step - loss: 1.4818e-07 - acc: 1.0000
Epoch 67/100
50/50 [==============================] - 24s 487ms/step - loss: 1.3535e-07 - acc: 1.0000
Epoch 68/100
50/50 [==============================] - 25s 502ms/step - loss: 1.3288e-07 - acc: 1.0000
Epoch 69/100
50/50 [==============================] - 26s 518ms/step - loss: 1.3219e-07 - acc: 1.0000
Epoch 70/100
50/50 [==============================] - 26s 511ms/step - loss: 1.3854e-07 - acc: 1.0000
Epoch 71/100
50/50 [==============================] - 25s 491ms/step - loss: 1.3212e-07 - acc: 1.0000
Epoch 72/100
50/50 [==============================] - 27s 536ms/step - loss: 1.3330e-07 - acc: 1.0000
Epoch 73/100
50/50 [==============================] - 28s 569ms/step - loss: 1.2946e-07 - acc: 

In [12]:
print(df3)

   Steps per Epoch  Epochs  Accuracy
0               10      10     0.875
1               10      20     0.750
2               10      30     0.000
3               30      10     0.000
4               30      20     0.000
5               30      30     0.250
6               50      10     0.375
7               50      20     0.000
8               50      30     0.125
9               50     100     0.375


4. Discuss the effect of the following on accuracy and loss (train & test): 

Increasing the steps_per_epoch
The steps_per_epoch refers the number of samples to yield from the generator before declaring an epoch to be completed. The more steps per epoch declared, the more accuracy should icrease and loss should decrease for the training data train and loss. If the model begins to overfit, accuracy will decrase and loss will increase for the training data set.  

Increasing the number of epochs
The number of epochs is indicative of the number of times a model will be trained by the entire data set.

5. Name two uses of zero padding in CNN.

If a user would like to keep the size of a particular image the same, they can use zero padding. Padding also imporves the performance of networks by keeping infromation at the boarders. 

6. What is the use of a 1 x 1 kernel in CNN? 

The uses of a 1x1 kernel in CNN is that it simply maps the first pixel with all the channels to an output level. This allows for dimensionality reduction and dimensionality increase.

7. What are the advantages of a CNN over a fully connected DNN for this image classification problem?

CNNs can utilize fewer paramteres because whenever a convolutional layer is used, the number of parameters are equivalent to the product of the dimesnions of that given filter multiplied by the total number of filters being calculated within that layer. It is very easy for a fully connected DNN total parameter count to explode if full connetions of all nodes are used throughout the layers.  